# 第5章 利用上下文信息

## 时间上下文信息

### 时间效应简介

时间信息对用户兴趣的影响表现在以下几个方面：
- **用户兴趣是变化的：**年龄的增长、工作年限的增加，如果要准确预测用户现在的兴趣，就应该关注用户最近的行为，因为用户最近的行为最能体现他现在的兴趣。
- **物品也是有生命周期的：**电影（物品）上映的时间，也可能受新闻事件的影响，当决定在某个时刻给某个用户推荐某个物品时，需要考虑该物品在该时刻是否已经过时了。
- **季节效应：**季节效应主要反应了时间本身对用户兴趣的影响，节日也是一种季节效应。

### 系统时间特性的分析
包含时间信息的用户行为数据集由一系列三元组构成，其中每个三元组$(u,i,t)$代表了用户$u$在时刻$t$对物品$i$产生过行为。
通过统计如下信息研究系统的时间特性：
- 数据集每天独立用户数的增长情况
- 系统的物品变化情况
- 用户访问情况

In [1]:
import os, sys
import pandas as pd

PROJECT_ROOT = os.path.dirname(sys.path[0])

In [2]:
user_bookmark_path = os.path.join(PROJECT_ROOT, 
                                  "src/data/delicious-2k/user_taggedbookmarks-timestamps.dat")
bookmarks_path = os.path.join(PROJECT_ROOT, 
                              "src/data/delicious-2k/bookmarks.dat")
user_bookmark_dataset = pd.read_table(user_bookmark_path, sep='\t', engine='python')
bookmarks_dataset = pd.read_table(bookmarks_path, sep='\t', engine='python')

bookmarks_dataset.rename(columns={'id':'bookmarkID'},inplace=True)
dataset = pd.merge(user_bookmark_dataset, bookmarks_dataset, how='left', on=['bookmarkID'])
dataset = dataset[['userID', 'bookmarkID', 'tagID', 'urlPrincipal', 'timestamp']]

dataset.head()

,userID,bookmarkID,tagID,urlPrincipal,timestamp
0,8,1,1,www.ifla.org,1289255362000
1,8,2,1,archive.ifla.org,1289255159000
2,8,7,1,www.edselect.com,1289238901000
3,8,7,6,www.edselect.com,1289238901000
4,8,7,7,www.edselect.com,1289238901000


In [3]:
def delete_tag_id(dataset, site_name):
    return dataset[dataset['urlPrincipal'].str.find(site_name) != -1]
                .drop(labels=['urlPrincipal', 'tagID'],axis=1).drop_duplicates()

nytimes_dataset = delete_tag_id(dataset, 'nytimes.com')
youtube_dataset = delete_tag_id(dataset, 'youtube.com')
wikipedia_dataset = delete_tag_id(dataset, 'wikipedia.com')
sourceforge_dataset = delete_tag_id(dataset, 'sourceforge.com')
blogspot_dataset = delete_tag_id(dataset, 'blogspot.com')

dataset_describe = []
dataset_describe.append((nytimes_dataset['userID'].nunique(), 
                         nytimes_dataset['bookmarkID'].nunique()))
dataset_describe.append((youtube_dataset['userID'].nunique(), 
                         youtube_dataset['bookmarkID'].nunique()))
dataset_describe.append((wikipedia_dataset['userID'].nunique(), 
                         wikipedia_dataset['bookmarkID'].nunique()))
dataset_describe.append((sourceforge_dataset['userID'].nunique(), 
                         sourceforge_dataset['bookmarkID'].nunique()))
dataset_describe.append((blogspot_dataset['userID'].nunique(), 
                         blogspot_dataset['bookmarkID'].nunique()))

pd.DataFrame(
        data = dataset_describe,
        index = ['nytimes', 'youtube', 'wikipedia', 'sourceforge', 'blogspot'],
        columns = ["user_total","item_total"]
)

,user_total,item_total
nytimes,546,877
youtube,711,1072
wikipedia,0,0
sourceforge,0,0
blogspot,989,1459


### 推荐系统的实时性

1. 实时推荐系统不能每天都给所有用户离线计算推荐结果，然后在线展示昨天计算出来的结果，所以，要求在每个用户访问推荐系统时，都根据用户这个时间点钱的行为**实时计算**推荐列表。
2. 推荐算法需要平衡考虑用户的近期行为和长期行为，既要让推荐列表反应出用户经期行为所体现的兴趣，又不能让推荐列表完全受用户近期行为的影响，要保证推荐列表对用户兴趣预测的**延续性**。

### 推荐算法的时间多样性

时间多样性：推荐系统每天推荐结果的变化程度。时间多样性高的推荐系统中，用户会经常看到不同的推荐结果。

**问题：**如何在不损失精度的情况下，提高推荐结果的时间多样性。  
提高推荐结果时间多样性需要分两步：  
1. 需要保证推荐系统能够在用户有了新的行为后，及时调整推荐结果。
2. 需要保证推荐系统在用户没有新的行为时，也能够经常变化一下结果。

解决如果用户没有行为，如何保证给用户的推荐结果具有一定的时间多样性：  
1. 在生成推荐结果时，加入一定的随机性。
2. 记录用户每天看到的推荐结果，然后在每天给用户进行推荐时，对他前几天看到过很多次的推荐结果进行适当地降权。
3. 每天给用户使用不同的推荐算法。

### 时间上下文推荐算法

#### 最近最热门

给定时间$T$、物品$i$最近的流行度$n_i(T)$定义为：$$n_i(T)=\sum_{(u,i,t) \in \text{Train}, t < T} \frac{1}{1 + \alpha(T-t)}$$其中，$\alpha$是时间衰减参数。

In [4]:
from main.util import delicious_reader

# 加载数据集
original_dataset = delicious_reader.load_data(bookmarks_path, user_bookmark_path)

# 对数据集进行划分
train_dataset, test_dataset = delicious_reader.split_data(
    delicious_reader.filter_dataset(original_dataset, "www.nytimes.com"))

In [5]:
from main.chapter5.recent_popularity import RecentPopular

# 训练模型
model = RecentPopular(train_dataset)
model.fit()

In [6]:
from main.util import delicious_reader, metric

def evaluate(model, test_dataset, N, K=None):
    """
    评估模型
    :param N: 推荐的商品个数
    :param K: 搜索邻近的用户个数
    :return: 精确率(precision), 召回率(recall)
    """
    test_dataset = delicious_reader.get_all_items(test_dataset)

    recommens = model.recommend_users(test_dataset.keys(), N=N, K=K)

    recall = metric.recall(recommends=recommens, tests=test_dataset)
    precision = metric.precision(recommends=recommens, tests=test_dataset)

    return precision, recall

In [7]:
metric_value = list()

N_list = [i for i in range(10, 110, 10)]

for N in N_list:
    single_eval = evaluate(model, test_dataset, N, K=None)
    metric_value.append(single_eval)
    
pd.DataFrame(
    data=metric_value,
    index=['N=' + str(i) for i in N_list],
    columns=["Precision", "Recall"]
)

,Precision,Recall
N=10,0.015801,0.001580
N=20,0.022573,0.001129
N=30,0.027088,0.000903
N=40,0.033860,0.000847
N=50,0.038375,0.000767
N=60,0.060948,0.001016
N=70,0.063205,0.000903
N=80,0.065463,0.000818
N=90,0.076749,0.000853
N=100,0.088036,0.000880


#### 时间上下文相关的ItemCF算法

&emsp;&emsp;基于物品的个性化推荐算法有两个核心部分构成：  
- 利用用户行为离线计算物品之间的相似度
- 根据用户的历史行为和物品相似度矩阵，给用户做在线个性化推荐。  

&emsp;&emsp;与时间信息相关的两个因素：  
- 物品相似度：用户在相隔很短的时间内喜欢的物品具有更高相似度
- 在线推荐：用户近期行为相比用户很久之前的行为，更能体现用户现在的兴趣。

&emsp;&emsp;基于物品的协同过滤算法，通过如下公式计算物品的相似度：$$\text{sim}(i,j)=\frac{\sum_{u \in N(i) \cap N(j)} 1 / \log(1 + |N(u)|)}{\sqrt{|N(i)||N(j)|}}$$&emsp;&emsp;而在给用户$u$做推荐时，用户$u$对物品$i$的兴趣$p(u,i)$表示为：$$p(u,i)=\sum_{j \in N(u)} \text{sim}(i,j)$$
&emsp;&emsp;在得到时间信息（用户对物品产生行为的时间）后，改进相似度计算公式：$$\text{sim}(i,j)=\frac{\sum_{u \in N(i) \cap N(j)} f(|t_{ui}-t_{uj}|)}{\sqrt{|N(i)||N(j)|}}$$&emsp;&emsp;其中$t_{ui}$是用户$u$对物品$i$产生行为的时间。$f$函数的含义是，用户对物品$i$和物品$j$产生行为的时间越远，则$f(|t_{ui}-t_{uj}|)$越小。  
衰减函数：$$f(|t_{ui}-t_{uj}|)=\frac{1}{1 + \alpha|t_{ui}-t_{uj}|}$$&emsp;&emsp;其中$\alpha$是时间衰减参数，如果一个系统用户兴趣变化越快，就应该取比较大的$\alpha$，反之需要取比较小的$\alpha$。

&emsp;&emsp;除了考虑时间信息对相关表的影响，也需要考虑时间信息对预测公式的影响，用户现在的行为应该和用户最近的行为关系更大，因此修正预测公式：$$p(u,i)=\sum_{j \in N(u) \cap S(i, K)} \text{sim}(i,j) \frac{1}{1 + \beta |t_0 - t_{uj}|}$$&emsp;&emsp;其中$t_0$是当前时间。  

&emsp;&emsp;公式含义：$t_{uj}$越靠近$t_0$，和物品$j$相似的物品就会在用户$u$的推荐列表中获得越高的排名，$\beta$是时间衰减参数。

In [8]:
from main.chapter5.tc_itemcf import TCItemCF

model = TCItemCF(train_dataset)
model.fit()

In [9]:
metric_value = list()
K = 10
N_list = [i for i in range(10, 110, 10)]

for N in N_list:
    single_eval = evaluate(model, test_dataset, N, K)
    metric_value.append(single_eval)

pd.DataFrame(
    data=metric_value,
    index=['N=' + str(i) for i in N_list],
    columns=["Precision", "Recall"]
)

,Precision,Recall
N=10,0.020316,0.056250
N=20,0.020316,0.052632
N=30,0.020316,0.052632
N=40,0.020316,0.052632
N=50,0.020316,0.052632
N=60,0.020316,0.052632
N=70,0.020316,0.052632
N=80,0.020316,0.052632
N=90,0.020316,0.052632
N=100,0.020316,0.052632


#### 时间上下文相关的UserCF算法

&emsp;&emsp;UserCF算法的基本思想：给用户推荐和他兴趣相似的其他用户喜欢的物品。  
&emsp;&emsp;与时间信息相关的两个因素：  
- 用户兴趣相似度：两个用户兴趣相似是因为他们喜欢相同的物品，或者对相同的物品产生过行为。如果两个用户**同时**喜欢相同的物品，那么他们应该有更大的兴趣相似度。
- 相似兴趣用户的最近行为：在找到和当前用户$u$兴趣相似的一组用户后，这组用户最近的兴趣，显然相比这组用户很久之前的兴趣，更加接近用户$u$今天的兴趣。

&emsp;&emsp;UserCF算法通过如下公式计算用户$u$和用户$v$的兴趣相似度：$$w_{uv}=\frac{|N(u) \cap N(v)|}{\sqrt{|N(u)||N(v)|}}$$&emsp;&emsp;其中$N(u)$是用户$u$喜欢的物品集合，$N(v)$是用户$v$喜欢的物品集合。  
&emsp;&emsp;考虑时间信息，改进公式如下：$$w_{uv}=\frac{\sum_{i \in N(u) \cap N(v)} \frac{1}{\displaystyle 1 + \alpha |t_{ui}-t_{vi}|}}{\sqrt{|N(u)||N(v)|}}$$
&emsp;&emsp;在得到用户相似度后，UserCF通过如下公式预测用户对物品的兴趣：$$p(u,i)=\sum_{v \in S(u,K)} w_{uv}r_{vi}$$&emsp;&emsp;其中，$S(u,K)$包含了和用户$u$兴趣最接近的$K$个用户。如果用户$v$对物品$i$产生过行为，那么$r_{vi}=1$，否则$r_{vi}=0$。  
&emsp;&emsp;如果考虑和用户$u$兴趣相似用户的最近兴趣，改进公式如下：$$p(u,i)=\sum_{v \in S(u,k)} w_{uv} r_{vi} \frac{1}{1+\alpha(t_0 - t_{vi})}$$

In [10]:
from main.chapter5.tc_usercf import TCUserCF

# 训练模型
model = TCUserCF(train_dataset)
model.fit()

In [11]:
metric_value = list()
K = 10
N_list = [i for i in range(10, 110, 10)]

for N in N_list:
    single_eval = evaluate(model, test_dataset, N, K)
    metric_value.append(single_eval)

pd.DataFrame(
    data=metric_value,
    index=['N=' + str(i) for i in N_list],
    columns=["Precision", "Recall"]
)

,Precision,Recall
N=10,0.018059,0.042553
N=20,0.018059,0.035088
N=30,0.020316,0.039130
N=40,0.020316,0.039130
N=50,0.020316,0.039130
N=60,0.020316,0.039130
N=70,0.020316,0.039130
N=80,0.020316,0.039130
N=90,0.020316,0.039130
N=100,0.020316,0.039130


### 时间段图模型

&emsp;&emsp;时间段图模型$G(U,S_U,I,S_I,E,w,\sigma)$是一个二分图。$U$是用户节点集合，$S_U$是用户时间段节点集合。一个用户时间段节点$v_ut \in S_U$会和用户$u$在时刻$t$喜欢的物品通过边相连。$I$是物品节点集合，$S_I$是物品时间段节点集合。一个物品时间段节点$v_it \in S_I$会和所有在时刻$t$喜欢物品$i$的用户通过边相连。$E$是边集合，$w(e)$定义了边的权重，$\sigma(e)$定义了顶点的权重。

图上两个相关性比较高的顶点一般具有如下特征：
- 两个顶点之间有很多路径相连
- 两个顶点之间的路径比较短
- 两个顶点之间的路径不经过出度比较大的顶点。

&emsp;&emsp;路径融合算法首先提取出两个顶点之间长度小于一个阈值的所有路径，然后根据每条路径经过的顶点，给每条路径赋予一定的权重，最后将两个顶点之间所有路径的权重之和作为连个顶点的相关度。

&emsp;&emsp;假设$P=\{v_1,v_2,\cdots,v_n\}$是连接顶点$v_1$和顶点$v_n$的一条路径，这条路径的权重$\Gamma(P)$取决于这条路径经过的所有顶点和变：$$\Gamma(P)=\sigma(v_n) \prod_{i=1}^{n-1} \frac{\sigma(v_i) \cdot w(v_i, v_{i+1})}{|\text{out}(v_i)|^\rho}$$其中，$\text{out}(v)$是顶点$v$指向的顶点集合，$|\text{out}(v)|$是顶点$v$的出度，$\sigma(v_i) \in (0,1]$定义了顶点的权重，$w(v_i,v_{i+1}) \in (0,1]$定义了边$e(v_i,v_{i+1})$的权重。  
&emsp;&emsp;对于顶点$v$和$v'$，令$p(v,v',K)$为这两个顶点间距离小于K的所有路径，这两个顶点之间的相关度可以定义为：$$d(v,v')=\sum_{P \in P(v,v', K)} \Gamma(P)$$  
&emsp;&emsp;对于时间段图模型，所有边的权重都定义为1，而顶点的权重$\sigma(v)$定义如下：$$\sigma(v) =\left \{ \begin{array}{cc} 
1 - \alpha \quad (v \in U) \\
\alpha \quad (v \in S_U) \\
1 - \beta \quad (v \in I) \\
\beta \quad (v \in S_I)
\end{array} \right.$$其中，$\alpha,\beta \in [0,1]$是两个参数，控制不同顶点的权重。

## 地点上下文信息

### 基于位置的推荐算法

LARS（位置感知推荐系统）将物品分成两类，一类是有空间属性的（餐馆、商店等），另一类是无空间属性的（图书、电影等），将用户也分成两类，一类是有空间属性的（用户现在的地址），另一类用户没有空间属性。
数据集有3种不同的形式：
- （用户，用户位置，物品，评分）
- （用户，物品，物品位置，评分）
- （用户，用户位置，物品，物品位置，评分）

### （用户，用户位置，物品，评分）数据集

&emsp;&emsp;LARS系统的基本思想是根据用户的位置划分成很多子集，因为位置信息是一个树状结构，数据集也会划分成一个树状结构。然后给定每个用户的位置，可以将他分配到某个叶子节点中，而该叶子节点包含了所有和他同一个位置的用户的行为数据集，利用这个叶子节点上的用户行为数据，采用ItemCF算法给用户进行推荐。

### （用户，物品，物品位置，评分）数据集
&emsp;&emsp;LARS系统会忽略物品的位置信息，利用ItemCF算法计算用户$u$对物品$i$的兴趣$P(u,i)$，但最终物品$i$在用户$u$的推荐列表中的权重定义为：$$\text{RecScore}(u,i) = P(u,i) - \text{TravelPenalty}(u,i)$$其中，TravelPenalty(u,i)表示了物品$i$的位置对用户$u$的代价。计算TravelPenalty(u,i)的基本思想是对于物品$i$与用户$u$之前评分的所有物品的位置计算距离的平均值（或者最小值）。  
&emsp;&emsp;为了避免计算用户对所有物品的TravelPenalty，LARS在计算用户$u$对物品$i$的兴趣度RecScore(u,i)时，首先对用户每一个曾经评过分的物品，找到和它距离小于阈值$d$的所有其他物品，然后将这些物品的集合作为候选集，然后再利用上面的公式计算最终的RecScore。